In [85]:
import pandas as pd
import networkx as nx
from karateclub import LabelPropagation, FeatherGraph, Diff2Vec, RandNE
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from node2vec import Node2Vec
from gensim.models import Word2Vec
import random
import matplotlib.pyplot as plt
import graphistry
from sklearn.metrics import accuracy_score, matthews_corrcoef, confusion_matrix, classification_report

In [ ]:
GRAPHISTRY_SERVER = "hub.graphistry.com"
GRAPHISTRY_USERNAME = "decentraliser"
GRAPHISTRY_PASSWORD = "nhr9BYE@dkb2dnz.prx"

graphistry.register(
        api=3,
        protocol="https",
        server=GRAPHISTRY_SERVER,
        username=GRAPHISTRY_USERNAME,
        password=GRAPHISTRY_PASSWORD,
    )

In [2]:
financial_transactions = pd.read_csv("data/financial_transactions.csv", index_col='_id')
non_financial_transactions = pd.read_csv("data/non_financial_transactions.csv",index_col='_id')

In [3]:
transactions = pd.concat([financial_transactions,non_financial_transactions])

In [4]:
transactions.timestamp = pd.to_datetime(transactions.timestamp, unit='s')

In [5]:
graph = nx.from_pandas_edgelist(transactions,'sender','receiver','value',nx.MultiDiGraph(),'timestamp')

In [7]:
graph = nx.convert_node_labels_to_integers(graph,label_attribute='address')
node_address_mapping = nx.get_node_attributes(graph, "address")

In [8]:
model = LabelPropagation()
model.fit(graph)

In [9]:
cluster_membership = model.get_memberships()

In [49]:
res = defaultdict(list)
for key, val in sorted(cluster_membership.items()):
    res[val].append(key)

In [169]:
size_dict = {key: len(res[key]) for key in res.keys()}
size_dict = dict(sorted(size_dict.items(), key=lambda item: item[1], reverse=True))

In [68]:
value_dict = {key: sum(dict(graph.in_degree(res[key], weight='value')).values()) for key in res.keys()}

In [72]:
value_dict = dict(sorted(value_dict.items(), key=lambda item: item[1], reverse=True))

In [ ]:
address_groups = {address: cluster_membership[node] for node, address in node_address_mapping.items()}

In [192]:
i = 17

In [193]:
cluster_subgraph = graph.subgraph(res[i])

In [194]:
address_cluster = list(map(node_address_mapping.get, res[i]))
print(len(address_cluster))

1449


In [195]:
node_address_mapping[i]

'0xFbdDaDD80fe7bda00B901FbAf73803F2238Ae655'

In [196]:
{"USDC": "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48", "USDT": "0xdAC17F958D2ee523a2206206994597C13D831ec7", "Uniswap": "0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45", "Opensea": "0x7f268357A8c2552623316e2562D90e642bB538E5", "WETH": "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2", "KuCoin": "0xD6216fC19DB775Df9774a6E33526131dA7D19a2c"}

{'USDC': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
 'USDT': '0xdAC17F958D2ee523a2206206994597C13D831ec7',
 'Uniswap': '0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45',
 'Opensea': '0x7f268357A8c2552623316e2562D90e642bB538E5',
 'WETH': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
 'KuCoin': '0xD6216fC19DB775Df9774a6E33526131dA7D19a2c'}

Can easily identify clusters and center them on large entities such as uniswap, all of these clusters essentially have the same structure with the main entity at the center